In [1]:
import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
import seaborn as sea
from scipy.stats import pearsonr
%matplotlib inline

In [2]:
#Carregando o dataset
df = pd.read_csv("analytics.csv", sep = ',', low_memory = False)

In [ ]:
df.head()

In [3]:
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['End Date'] = pd.to_datetime(df['End Date'])
df

Trip ID  Duration          Start Date  \
0      431032      1648 2014-08-29 15:33:00   
1      383548       617 2014-07-28 17:35:00   
2      257887       551 2014-04-22 19:19:00   
3      334581       327 2014-06-22 19:36:00   
4      268757       382 2014-05-01 19:37:00   
...       ...       ...                 ...   
90226  216465      1178 2014-03-17 08:54:00   
90227  290969       120 2014-05-20 08:34:00   
90228  324829      1807 2014-06-15 12:08:00   
90229  266083       522 2014-04-30 08:35:00   
90230  367598       255 2014-07-17 08:22:00   

                                 Start Station  Start Terminal  \
0                            Steuart at Market              74   
1                            Steuart at Market              74   
2                               Post at Kearny              47   
3                                5th at Howard              57   
4                              Clay at Battery              41   
...                                        ...             ...   
90226                          Townsend at 7th              65   
90227                        Steuart at Market              74   
90228  San Francisco Caltrain 2 (330 Townsend)              69   
90229                         Davis at Jackson              42   
90230                 Commercial at Montgomery              45   

                 End Date                                    End Station  \
0     2014-08-29 16:00:00       San Francisco Caltrain (Townsend at 4th)   
1     2014-07-28 17:45:00       San Francisco Caltrain (Townsend at 4th)   
2     2014-04-22 19:28:00                       South Van Ness at Market   
3     2014-06-22 19:42:00                                  2nd at Folsom   
4     2014-05-01 19:44:00                                  Market at 4th   
...                   ...                                            ...   
90226 2014-03-17 09:14:00                         Embarcadero at Vallejo   
90227 2014-05-20 08:36:00                          Embarcadero at Folsom   
90228 2014-06-15 12:38:00                         Embarcadero at Vallejo   
90229 2014-04-30 08:44:00                                  Market at 4th   
90230 2014-07-17 08:26:00  Temporary Transbay Terminal (Howard at Beale)   

       End Terminal  Bike # Subscriber Type Zip Code  
0                70     411      Subscriber    94107  
1                70     259      Subscriber    94403  
2                66     414      Subscriber    94105  
3                62     553      Subscriber    94103  
4                76     501      Subscriber    94612  
...             ...     ...             ...      ...  
90226            48     478      Subscriber    94107  
90227            51     422      Subscriber    94044  
90228            48     379        Customer    94041  
90229            76     347      Subscriber    94111  
90230            55     592      Subscriber    94939  

[90231 rows x 11 columns]

In [6]:
df['Start Day'] = df['Start Date'].dt.floor('h')
df['End Day'] = df['End Date'].dt.floor('h')
df.head()

Trip ID  Duration          Start Date      Start Station  Start Terminal  \
0  431032      1648 2014-08-29 15:33:00  Steuart at Market              74   
1  383548       617 2014-07-28 17:35:00  Steuart at Market              74   
2  257887       551 2014-04-22 19:19:00     Post at Kearny              47   
3  334581       327 2014-06-22 19:36:00      5th at Howard              57   
4  268757       382 2014-05-01 19:37:00    Clay at Battery              41   

             End Date                               End Station  End Terminal  \
0 2014-08-29 16:00:00  San Francisco Caltrain (Townsend at 4th)            70   
1 2014-07-28 17:45:00  San Francisco Caltrain (Townsend at 4th)            70   
2 2014-04-22 19:28:00                  South Van Ness at Market            66   
3 2014-06-22 19:42:00                             2nd at Folsom            62   
4 2014-05-01 19:44:00                             Market at 4th            76   

   Bike # Subscriber Type Zip Code           Start Day             End Day  
0     411      Subscriber    94107 2014-08-29 15:00:00 2014-08-29 16:00:00  
1     259      Subscriber    94403 2014-07-28 17:00:00 2014-07-28 17:00:00  
2     414      Subscriber    94105 2014-04-22 19:00:00 2014-04-22 19:00:00  
3     553      Subscriber    94103 2014-06-22 19:00:00 2014-06-22 19:00:00  
4     501      Subscriber    94612 2014-05-01 19:00:00 2014-05-01 19:00:00

In [23]:
trips_start_count = df[['Trip ID', 'Start Day', 'Start Station']].groupby(['Start Day','Start Station']).agg(['count']).reset_index()
trips_end_count = df[['Trip ID', 'End Day', 'End Station']].groupby(['End Day','End Station']).agg(['count']).reset_index()
trips_end_count.rename(columns={"End Day": "Day", "End Station": "Station"}, inplace = True)
trips_start_count.rename(columns={"Start Day": "Day", "Start Station": "Station"}, inplace = True)
trips_start_count.head()

Day                                   Station Trip ID
                                                                  count
0 2014-03-01 00:00:00         Civic Center BART (7th at Market)       1
1 2014-03-01 00:00:00                        Powell Street BART       1
2 2014-03-01 00:00:00             Redwood City Caltrain Station       1
3 2014-03-01 00:00:00  San Francisco Caltrain (Townsend at 4th)       1
4 2014-03-01 01:00:00                 Broadway St at Battery St       1

In [44]:
df_inner = pd.merge(trips_start_count, trips_end_count, on=['Day', 'Station'], how='inner')
df_inner.head()

Day                               Station Trip ID_x  \
                                                                count   
0 2014-03-01 03:00:00  Harry Bridges Plaza (Ferry Building)         1   
1 2014-03-01 08:00:00                        Market at 10th         1   
2 2014-03-01 09:00:00                Embarcadero at Vallejo         2   
3 2014-03-01 09:00:00  Harry Bridges Plaza (Ferry Building)         2   
4 2014-03-01 10:00:00                Embarcadero at Sansome         1   

  Trip ID_y  
      count  
0         3  
1         2  
2         1  
3         2  
4         4

In [42]:
df_inner.columns = df_inner.columns.get_level_values(0)

In [49]:
df_inner['balance'] = df_inner['Trip ID_x'] - df_inner['Trip ID_y']
df_inner.head()

Day                               Station Trip ID_x  \
                                                                count   
0 2014-03-01 03:00:00  Harry Bridges Plaza (Ferry Building)         1   
1 2014-03-01 08:00:00                        Market at 10th         1   
2 2014-03-01 09:00:00                Embarcadero at Vallejo         2   
3 2014-03-01 09:00:00  Harry Bridges Plaza (Ferry Building)         2   
4 2014-03-01 10:00:00                Embarcadero at Sansome         1   

  Trip ID_y balance  
      count          
0         3      -2  
1         2      -1  
2         1       1  
3         2       0  
4         4      -3

In [50]:
df_inner.columns = df_inner.columns.get_level_values(0)

In [52]:
df_inner.to_csv('bikes_balance.csv')